In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import os
from datetime import datetime
from typing import Any, List
import pandas as pd
import globus_sdk
from globus_sdk import AccessTokenAuthorizer, TransferClient, TransferData
from globus_sdk.scopes import TransferScopes
from attrs import define, field, Factory

from phoglobushelpers.PhoGlobusHelper import GlobusConnector, KnownEndpoints
from phoglobushelpers.compatibility_objects.Bookmarks import Bookmark, BookmarkList
from phoglobushelpers.compatibility_objects.Files import File, FilesystemDataType, FileList
from phoglobushelpers.compatibility_objects.Tasks import FatalError, Task, TaskList

Automatic pdb calling has been turned OFF


In [2]:
connect_man = GlobusConnector.login_and_get_transfer_client()
transfer_client = connect_man.transfer_client
connect_man.list_endpoints()


Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=769d24e1-d1cc-4198-9ff7-2626485da449&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=eDeW1X5L7JWKOB2jFZ0325FmpMSO4DXQZUAdNqUybIk&code_challenge_method=S256&access_type=offline
	 Copied url to clipboard!
My Endpoints:
[c3a90494-2555-11ec-a47d-a50ad076c282] aleinbook-linux
[84991054-07b4-11ed-8d83-a54cf61939f8] Apogee
[6d0251c4-2585-11ec-9e35-3df4ed83d858] Cubix Win10
[c3578f36-42a8-11ec-a50f-b537d6c07c1d] FlatEdge_Server
[af3fcfce-f664-11ed-9a7d-83ef71fbf0ae] LNX00052_Fedora
[20c84240-1eb1-11eb-81b7-0e2f230cc907] Pho Personal Laptop
[560d3a12-be20-11ed-9916-cb2cff506ca5] Pho_Personal_Testix
[debb635c-2556-11ec-a47d-a50ad076c282] rMBP Pink Dot
[e414f584-2556-11ec-a0a7-6b21ca6daf73] rMBP Pink Dot


In [3]:
task_id = "3bc6e65c-24f1-11ee-abf4-63e0d97254cd"

In [5]:
tasks_list = connect_man.get_tasks()
tasks_list_df = tasks_list.to_dataframe()


In [6]:
tasks_list_df

,bytes_checksummed,bytes_transferred,canceled_by_admin,canceled_by_admin_message,command,completion_time,deadline,delete_destination_extra,destination_endpoint,destination_endpoint_display_name,...,subtasks_retrying,subtasks_skipped_errors,subtasks_succeeded,subtasks_total,symlinks,sync_level,task_id,type,username,verify_checksum
0,1534112401584,0,None,None,API 0.10,2023-04-20T21:12:02+00:00,2023-04-20T21:11:03+00:00,False,u_gjz3ny5efnehvgt65z3lxdqd74#b82d3b90-7b07-11e...,UMich ARC Non-Sensitive Data Den Volume Collec...,...,0,0,7024,60551,0,3.0,6e5e35fc-def6-11ed-9b9b-c9bb788c490e,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
1,14225265,0,None,None,API 0.10,2023-04-20T08:46:58+00:00,2023-04-21T08:46:52+00:00,False,None,None,...,0,0,553,553,0,3.0,e540ea28-df57-11ed-9a5f-83ef71fbf0ae,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
2,14225265,0,None,None,API 0.10,2023-04-20T12:46:57+00:00,2023-04-21T12:46:52+00:00,False,None,None,...,0,0,553,553,0,3.0,6c5b84de-df79-11ed-9b9b-c9bb788c490e,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
3,14225120,17843,None,None,API 0.10,2023-04-20T16:47:29+00:00,2023-04-21T16:46:52+00:00,False,None,None,...,0,0,568,568,0,3.0,f331b4ee-df9a-11ed-9a5f-83ef71fbf0ae,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
4,14231412,18769,None,None,API 0.10,2023-04-20T20:47:27+00:00,2023-04-21T20:46:52+00:00,False,None,None,...,0,0,568,568,0,3.0,7a85a862-dfbc-11ed-9b9b-c9bb788c490e,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
5,0,1156761587,None,None,API 0.10 go,2023-04-20T22:27:33+00:00,2023-04-21T22:20:07+00:00,False,None,None,...,0,0,39,39,0,NaN,81463894-dfc9-11ed-9a5f-83ef71fbf0ae,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
6,0,0,None,None,API 0.10,2023-04-20T23:31:43+00:00,2023-04-21T23:27:16+00:00,False,u_gjz3ny5efnehvgt65z3lxdqd74#b82d3b90-7b07-11e...,UMich ARC Non-Sensitive Data Den Volume Collec...,...,0,0,3191,3191,0,0.0,e2b6fc68-dfd2-11ed-9b9b-c9bb788c490e,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
7,0,0,None,None,API 0.10,2023-04-20T23:52:53+00:00,2023-04-21T23:27:56+00:00,False,u_gjz3ny5efnehvgt65z3lxdqd74#b82d3b90-7b07-11e...,UMich ARC Non-Sensitive Data Den Volume Collec...,...,0,0,7696,7696,0,0.0,fa655c60-dfd2-11ed-ba2c-09d6a6f08166,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
8,0,0,None,None,API 0.10 go,2023-04-20T23:32:49+00:00,2023-04-21T23:28:11+00:00,False,u_gjz3ny5efnehvgt65z3lxdqd74#b82d3b90-7b07-11e...,UMich ARC Non-Sensitive Data Den Volume Collec...,...,0,0,8287,8287,0,0.0,035c28c6-dfd3-11ed-ba2c-09d6a6f08166,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True
9,0,62220,None,None,API 0.10 go,2023-04-20T23:48:19+00:00,2023-04-21T23:28:35+00:00,False,u_gjz3ny5efnehvgt65z3lxdqd74#b82d3b90-7b07-11e...,UMich ARC Non-Sensitive Data Den Volume Collec...,...,0,0,4377,4377,0,0.0,11b877f8-dfd3-11ed-ba2c-09d6a6f08166,TRANSFER,u_upma7q3dzfeqzg454wkb7lyqe4,True


In [3]:
# [Bookmark(bookmark_id='9a8eccc8-2558-11ec-a0a7-6b21ca6daf73', name='aleinbook-linux - PhoPy3DPositionAnalysis', endpoint_id='c3a90494-2555-11ec-a47d-a50ad076c282', path='/~/repo/PhoPy3DPositionAnalysis/'),
#  Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/'),
#  Bookmark(bookmark_id='caf6bf20-9c14-11ed-a29d-8383522b48d9', name='Apogee greatlakes_halechr Folder', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/Data/greatlakes_halechr/'),
#  Bookmark(bookmark_id='e1ef1556-4985-11ed-89d5-ede5bae4f491', name='Apogee Personal Dropbox Active', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/E/Dropbox (Personal)/Active/'),
#  Bookmark(bookmark_id='63655524-164e-11ee-bfba-195c41bc0be4', name='Apogee - PersonalFastSwapShare', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/FastSwap/PersonalFastSwapShare/'),
#  Bookmark(bookmark_id='1c725da6-203a-11ee-80bc-a3018385fcef', name='Apogee - ProgrammaticDisplayFunctionTesting', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/C/Users/pho/repos/Spike3DWorkEnv/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='4152f10a-a17b-11ed-a2a4-8383522b48d9', name='Apogee R VMs', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/R/FastSwap/Virtual Machines/'),
#  Bookmark(bookmark_id='78ca952e-2586-11ec-9e35-3df4ed83d858', name='Cubix Win10 PhoPy3DPositionAnalysis2021', endpoint_id='6d0251c4-2585-11ec-9e35-3df4ed83d858', path='/~/repos/PhoPy3DPositionAnalysis2021/'),
#  Bookmark(bookmark_id='e5c8531c-3b77-11ec-adf2-0748e6f5ffe3', name='Cubix Win10 - PhoPy3DPositionAnalysis2021 - data', endpoint_id='6d0251c4-2585-11ec-9e35-3df4ed83d858', path='/~/repos/PhoPy3DPositionAnalysis2021/data/'),
#  Bookmark(bookmark_id='3844ee50-9826-11ed-a849-256017f36728', name='Diba Lab Workstation 2022 - Cloud', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/home/halechr/cloud/'),
#  Bookmark(bookmark_id='3995215e-b76a-11ed-a982-5f0e34a3cc4f', name='Diba Lab Workstation 2022 - Cloud - MED-DibaLabDropbox', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/home/halechr/cloud/Dropbox_Diba_Shared/MED-DibaLabDropbox/'),
#  Bookmark(bookmark_id='561e4ad0-b76a-11ed-a982-5f0e34a3cc4f', name='Diba Lab Workstation 2022 - Cloud - MED-DibaLabDropbox/Data/', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/home/halechr/cloud/Dropbox_Diba_Shared/MED-DibaLabDropbox/Data/'),
#  Bookmark(bookmark_id='ba158112-dfd2-11ed-9b9b-c9bb788c490e', name='Diba Lab Workstation 2022 - GDrive Diba Shared', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/media/MAX/cloud/GDrive_Diba_Shared/Data/'),
#  Bookmark(bookmark_id='55246d18-9ce3-11ed-b577-33287ee02ec7', name='Diba Lab Workstation 2022 - greatlakes homedir', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/media/MAX/greatlakes_halechr/'),
#  Bookmark(bookmark_id='f7bd6c36-9825-11ed-a849-256017f36728', name='Diba Lab Workstation 2022 - MAX', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/media/MAX/Data/'),
#  Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/'),
#  Bookmark(bookmark_id='8ce139f8-9d46-11ed-a2a2-8383522b48d9', name='KDIBA Lab Turbo', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/'),
#  Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/'),
#  Bookmark(bookmark_id='248a9a30-164e-11ee-bfba-195c41bc0be4', name='LNX00052_Fedora - MAX', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/media/MAX/'),
#  Bookmark(bookmark_id='af68b1ea-20c7-11ee-818f-195c41bc0be4', name='LNX00052_Fedora - ProgrammaticDisplayFunctionTesting', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/home/halechr/repo/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='034e1614-3c19-11ec-90c8-7fa38c35632a', name='NYU Langone Health DTN1 - Buzsakilab Public Datasets', endpoint_id='188a6110-96db-11eb-b7a9-f57b2d55370d', path='/'),
#  Bookmark(bookmark_id='0bc636fc-3997-11ec-9f1c-db52857db67d', name='Pho Personal Laptop - Hiro_Datasets', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Volumes/iNeo/Data/Rotation_3_Kamran Diba Lab/DataProcessingProject/'),
#  Bookmark(bookmark_id='1805df24-e618-11ed-9a63-83ef71fbf0ae', name='Pho Personal Laptop - PegasusR6 - Data', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Volumes/Pegasus R6/Data/'),
#  Bookmark(bookmark_id='bc4aa210-3b77-11ec-adf2-0748e6f5ffe3', name='Pho Personal Laptop - PhoMatlabDataScripting Exported Data', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/~/repo/Python Projects/PhoNeuronGillespie2021CodeRepo/PhoMatlabDataScripting/'),
#  Bookmark(bookmark_id='8f77467e-2558-11ec-a0a7-6b21ca6daf73', name='Pho Personal Laptop - PhoPy3DPositionAnalysis', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/~/repo/Python Projects/PhoPy3DPositionAnalysis/'),
#  Bookmark(bookmark_id='99ce1e08-bdf1-11ed-8cec-f9fa098153fc', name='UMich Diba Turbo - Pho Directory', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Pho/'),
#  Bookmark(bookmark_id='b1668f08-2036-11ee-80bc-a3018385fcef', name='umich#greatlakes - halechr homedir', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/'),
#  Bookmark(bookmark_id='b0569b9c-2558-11ec-a0a7-6b21ca6daf73', name='umich#greatlakes - halechr home folder', endpoint_id='e0370902-9f48-11e9-821b-02b7a92d8e58', path='/~/'),
#  Bookmark(bookmark_id='f479daf4-2039-11ee-80bc-a3018385fcef', name='umich#greatlakes - ProgrammaticDisplayFunctionTesting', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='bef81650-96c5-11ed-9b93-19370d280681', name='umms-dibalab DataDen', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/')]

bookmark_list = connect_man.get_bookmarks()
bookmark_list

BookmarkList(DATA_TYPE='bookmark_list', DATA=[Bookmark(bookmark_id='9a8eccc8-2558-11ec-a0a7-6b21ca6daf73', name='aleinbook-linux - PhoPy3DPositionAnalysis', endpoint_id='c3a90494-2555-11ec-a47d-a50ad076c282', path='/~/repo/PhoPy3DPositionAnalysis/'), Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/'), Bookmark(bookmark_id='caf6bf20-9c14-11ed-a29d-8383522b48d9', name='Apogee greatlakes_halechr Folder', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/Data/greatlakes_halechr/'), Bookmark(bookmark_id='e1ef1556-4985-11ed-89d5-ede5bae4f491', name='Apogee Personal Dropbox Active', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/E/Dropbox (Personal)/Active/'), Bookmark(bookmark_id='63655524-164e-11ee-bfba-195c41bc0be4', name='Apogee - PersonalFastSwapShare', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/FastSwap/PersonalFastSwapShare/'), Bookmark(bookmar

In [4]:
lab_Turbo_data_folder_bookmark = Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')
lab_Turbo_data_folder_bookmark

Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')

In [5]:
lab_DataDen_data_folder_bookmark = Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/')
lab_DataDen_data_folder_bookmark

Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/')

In [6]:
# target_bookmark: Bookmark = lab_DataDen_data_folder_bookmark
target_bookmark: Bookmark = lab_Turbo_data_folder_bookmark
target_bookmark

Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')

In [28]:
file_list = connect_man.list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date=None, end_date=None)
file_list.to_dataframe()

,type,parent_path,name,last_modified,size
0,file,/umms-kdiba/Data/,across_session_result_handler_2023-07-21.pkl,2023-07-21 23:46:26+00:00,93597323
1,file,/umms-kdiba/Data/,across_session_result_long_short_inst_firing_r...,2023-07-14 14:20:27+00:00,63934595
2,file,/umms-kdiba/Data/,across_session_result_long_short_inst_firing_r...,2023-07-15 01:16:42+00:00,124697996
3,file,/umms-kdiba/Data/,across_session_result_long_short_inst_firing_r...,2023-07-19 04:12:51+00:00,150235843
4,file,/umms-kdiba/Data/,across_session_result_long_short_inst_firing_r...,2023-07-19 19:51:17+00:00,150236342
5,file,/umms-kdiba/Data/,across_session_result_long_short_inst_firing_r...,2023-07-21 21:15:58+00:00,93597323
6,file,/umms-kdiba/Data/,across_session_result_long_short_inst_firing_r...,2023-08-05 00:39:06+00:00,98338047
7,file,/umms-kdiba/Data/,across_session_result_long_short_inst_firing_r...,2023-08-07 20:29:52+00:00,98471212
8,file,/umms-kdiba/Data/,global_batch_output_2023-08-04.h5,2023-08-05 00:36:29+00:00,64237544
9,file,/umms-kdiba/Data/,global_batch_output_2023-08-07.h5,2023-08-07 20:25:46+00:00,64237544


In [23]:
recent_file_list = connect_man.list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date="2023-08-07", end_date=None, should_list_recursively=False)
recent_file_list.to_dataframe() #.columns

,type,parent_path,name,last_modified,size
0,file,/umms-kdiba/Data/,across_session_result_long_short_inst_firing_r...,2023-08-07 20:29:52+00:00,98471212
1,file,/umms-kdiba/Data/,global_batch_output_2023-08-07.h5,2023-08-07 20:25:46+00:00,64237544
2,file,/umms-kdiba/Data/,global_batch_result_2023-08-07.pkl,2023-08-07 20:25:24+00:00,99359262


In [29]:
recent_file_list = connect_man.list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date="2023-08-07", end_date=None, should_list_recursively=True)
recent_file_list # 1m 44.7 sec

FileList(DATA=[File(group='umms-kdiba-turbo', last_modified='2023-08-07 20:12:25+00:00', link_group=None, link_last_modified=None, link_size=None, link_target=None, link_user=None, name='pipeline_results.h5', permissions='0770', size=527170839, type=<FilesystemDataType.FILE: 'file'>, user='halechr', parent_path='/umms-kdiba/Data//KDIBA/gor01/one/2006-6-08_14-26-15/output'), File(group='umms-kdiba-turbo', last_modified='2023-08-07 20:11:57+00:00', link_group=None, link_last_modified=None, link_size=None, link_target=None, link_user=None, name='pipeline_results.h5', permissions='0770', size=465614187, type=<FilesystemDataType.FILE: 'file'>, user='halechr', parent_path='/umms-kdiba/Data//KDIBA/gor01/one/2006-6-09_1-22-43/output'), File(group='umms-kdiba-turbo', last_modified='2023-08-07 20:11:16+00:00', link_group=None, link_last_modified=None, link_size=None, link_target=None, link_user=None, name='pipeline_results.h5', permissions='0770', size=188066635, type=<FilesystemDataType.FILE: '

In [30]:
recent_file_list_df = recent_file_list.to_dataframe()
recent_file_list_df

,type,parent_path,name,last_modified,size
0,file,/umms-kdiba/Data//KDIBA/gor01/one/2006-6-08_14...,pipeline_results.h5,2023-08-07 20:12:25+00:00,527170839
1,file,/umms-kdiba/Data//KDIBA/gor01/one/2006-6-09_1-...,pipeline_results.h5,2023-08-07 20:11:57+00:00,465614187
2,file,/umms-kdiba/Data//KDIBA/gor01/one/2006-6-12_15...,pipeline_results.h5,2023-08-07 20:11:16+00:00,188066635
3,file,/umms-kdiba/Data//KDIBA/gor01/two/2006-6-07_16...,pipeline_results.h5,2023-08-07 20:12:19+00:00,520531010
4,file,/umms-kdiba/Data//KDIBA/gor01/two/2006-6-08_21...,pipeline_results.h5,2023-08-07 20:11:23+00:00,341153735
5,file,/umms-kdiba/Data//KDIBA/gor01/two/2006-6-09_22...,pipeline_results.h5,2023-08-07 20:13:15+00:00,643939306
6,file,/umms-kdiba/Data//KDIBA/gor01/two/2006-6-12_16...,pipeline_results.h5,2023-08-07 20:11:08+00:00,168391741
7,file,/umms-kdiba/Data//KDIBA/pin01/one/11-02_17-46-...,pipeline_results.h5,2023-08-07 20:13:36+00:00,202925192
8,file,/umms-kdiba/Data//KDIBA/pin01/one/11-02_19-28-...,pipeline_results.h5,2023-08-07 20:14:07+00:00,150551944
9,file,/umms-kdiba/Data//KDIBA/pin01/one/11-03_12-3-2...,pipeline_results.h5,2023-08-07 20:14:07+00:00,100341583


In [30]:
recent_file_list.path

'/umms-kdiba/Data/'

In [25]:
recent_file_list = connect_man.official_list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date="2023-08-07", end_date=None, should_list_recursively=True)
recent_file_list # 1m 44.7 sec
recent_file_list_df = recent_file_list.to_dataframe()
recent_file_list_df

FileList(DATA=[File(group='umms-kdiba-turbo', last_modified='2023-08-07 20:29:52+00:00', link_group=None, link_last_modified=None, link_size=None, link_target=None, link_user=None, name='across_session_result_long_short_inst_firing_rate_2023-08-07.pkl', permissions='0770', size=98471212, type=<FilesystemDataType.FILE: 'file'>, user='halechr', parent_path='/umms-kdiba/Data/'), File(group='umms-kdiba-turbo', last_modified='2023-08-07 20:25:46+00:00', link_group=None, link_last_modified=None, link_size=None, link_target=None, link_user=None, name='global_batch_output_2023-08-07.h5', permissions='0770', size=64237544, type=<FilesystemDataType.FILE: 'file'>, user='halechr', parent_path='/umms-kdiba/Data/'), File(group='umms-kdiba-turbo', last_modified='2023-08-07 20:25:24+00:00', link_group=None, link_last_modified=None, link_size=None, link_target=None, link_user=None, name='global_batch_result_2023-08-07.pkl', permissions='0770', size=99359262, type=<FilesystemDataType.FILE: 'file'>, user

In [31]:
recent_file_list.build_file_paths()

['/umms-kdiba/Data//KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/gor01/two/2006-6-07_16-40-19/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/gor01/two/2006-6-08_21-16-25/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/gor01/two/2006-6-09_22-24-40/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/gor01/two/2006-6-12_16-53-46/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/pin01/one/11-02_17-46-44/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/pin01/one/11-02_19-28-0/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/pin01/one/11-03_12-3-25/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/pin01/one/fet11-01_12-58-54/output/pipeline_results.h5',
 '/umms-kdiba/Data//KDIBA/vvp01/one/2006-4-09_17-29-30/output/pipeline_results.h5',
 '/umms-